In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import fmin, tpe, hp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Data/test.csv')

In [ ]:
train = pd.concat([train, test])

In [ ]:
train.shape

(105000, 16)

In [ ]:
def normalizeCol(col):
  max = col.max()
  min = col.min()
  diff = max-min
  for x in range(len(col)):
    col[x] = (col[x]-min)/diff

In [ ]:
columns = ['Name', 'Roll No']
train.drop(columns=columns, axis=1, inplace=True)

In [ ]:
train.head()

,Caoching,High School Marks,12th Appearing,Intermediate Marks,Income,Dummy,School Board,Device to Study,Online Course,Category,PwD,Age,Sex,Passed
0,NO,80.5,NO,92.2,59000,NO,Maharashtra Board,Laptop,YES,General-EWS,NO,20,M,FAIL
1,NO,83.3,YES,NaN,390000,NO,BSEB,Phone,YES,ST,NO,18,F,FAIL
2,NO,76.4,YES,NaN,3140000,NO,CBSE,NaN,NaN,OBC-NCL,NO,18,M,FAIL
3,YES,89.7,NO,79.4,345000,YES,HP Board,NaN,NaN,SC,NO,21,M,PASS
4,YES,98.9,NO,95.3,29000,NO,CBSE,NaN,NaN,SC,YES,17,M,PASS


In [ ]:
cols = ['Caoching', '12th Appearing', 'Dummy', 'School Board', 'Device to Study', 'Online Course', 'Category', 'PwD', 'Sex', 'Passed']

In [ ]:
label_encoder = LabelEncoder()
for i in cols:
  train[i] = label_encoder.fit_transform(train[i])

In [ ]:
train.head(5)

,Caoching,High School Marks,12th Appearing,Intermediate Marks,Income,Dummy,School Board,Device to Study,Online Course,Category,PwD,Age,Sex,Passed
0,0,80.5,0,92.2,59000,0,13,0,0,1,0,20,1,0
1,0,83.3,1,NaN,390000,0,1,1,0,5,0,18,0,0
2,0,76.4,1,NaN,3140000,0,2,3,1,3,0,18,1,0
3,1,89.7,0,79.4,345000,1,6,3,1,4,0,21,1,1
4,1,98.9,0,95.3,29000,0,2,3,1,4,1,17,1,1


In [ ]:
for col in train.columns:
  print(col, train[col].isnull().values.any())

Caoching False
High School Marks False
12th Appearing False
Intermediate Marks True
Income False
Dummy False
School Board False
Device to Study False
Online Course False
Category False
PwD False
Age False
Sex False
Passed False


In [ ]:
train['Intermediate Marks'].fillna(np.mean(train['Intermediate Marks']), inplace=True)

In [ ]:
train.head()

,Caoching,High School Marks,12th Appearing,Intermediate Marks,Income,Dummy,School Board,Device to Study,Online Course,Category,PwD,Age,Sex,Passed
0,0,80.5,0,92.200000,59000,0,13,0,0,1,0,20,1,0
1,0,83.3,1,87.428725,390000,0,1,1,0,5,0,18,0,0
2,0,76.4,1,87.428725,3140000,0,2,3,1,3,0,18,1,0
3,1,89.7,0,79.400000,345000,1,6,3,1,4,0,21,1,1
4,1,98.9,0,95.300000,29000,0,2,3,1,4,1,17,1,1


In [ ]:
test = train[100000:]
train = train[:100000]

In [ ]:
# for col in ['High School Marks', 'Intermediate Marks', 'Income', 'Age']:
#   normalizeCol(train[col])
#   normalizeCol(test[col])
columns_to_scale = ['High School Marks', 'Intermediate Marks', 'Income', 'Age']
scaler = StandardScaler()
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
test[columns_to_scale] = scaler.fit_transform(test[columns_to_scale])

In [ ]:
train.head()

,Caoching,High School Marks,12th Appearing,Intermediate Marks,Income,Dummy,School Board,Device to Study,Online Course,Category,PwD,Age,Sex,Passed
0,0,-0.960002,0,0.849300,-0.438280,0,13,0,0,1,0,0.876842,1,0
1,0,-0.574427,1,0.001002,-0.063489,0,1,1,0,5,0,-0.292577,0,0
2,0,-1.524593,1,0.001002,3.050335,0,2,3,1,3,0,-0.292577,1,0
3,1,0.306886,0,-1.426446,-0.114443,1,6,3,1,4,0,1.461551,1,1
4,1,1.573774,0,1.400457,-0.472249,0,2,3,1,4,1,-0.877286,1,1


In [ ]:
# rf = RandomForestClassifier()
# svc = SVC(kernel='rbf', random_state=69)
# lr = LogisticRegression()
xgb = XGBClassifier()

In [ ]:
labelCol = ['Passed']
X = train[train.columns[~train.columns.isin(labelCol)]]
Y = train[labelCol]
x_test = test[test.columns[~test.columns.isin(labelCol)]]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state = 0)

In [ ]:
# rf.fit(X_train, Y_train)
# svc.fit(X_train, Y_train)
# lr.fit(X_train, Y_train)
xgb.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
accuracy_score(xgb.predict(X_test), Y_test)

0.96885

In [ ]:
# preds_rf = rf.predict(x_test)
# preds_svc = svc.predict(x_test)
# preds_lr = lr.predict(x_test)
preds_xgb = xgb.predict(x_test)

In [ ]:
accuracy_score(preds_xgb, Y_test)

0.96844

In [ ]:
roll_numbers_list = pd.read_csv('/content/drive/MyDrive/Data/test.csv')['Roll No'].tolist()

In [ ]:
data = {
    'Roll No': roll_numbers_list,
    'Pass': preds_xgb.tolist(),
}

In [ ]:
result = pd.DataFrame(data)

In [ ]:
result.head()

,Roll No,Pass
0,546722345,1
1,546722346,1
2,546722347,1
3,546722348,0
4,546722349,1


In [ ]:
result['Pass'][result['Pass'] == 0] = 'FAIL'
result['Pass'][result['Pass'] == 1] = 'PASS'

In [ ]:
result.head()

,Roll No,Pass
0,546722345,PASS
1,546722346,PASS
2,546722347,PASS
3,546722348,FAIL
4,546722349,PASS


In [ ]:
result.to_csv('result.csv', index=False)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(128, 32, 16),
                    max_iter=10000, activation = 'logistic', random_state=42)

In [ ]:
mlp.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='logistic', hidden_layer_sizes=(128, 32, 16),
              max_iter=10000, random_state=42)

In [ ]:
y_pred = mlp.predict(X_test)

In [ ]:
accuracy_score(Y_test, y_pred)

0.9714

In [ ]:
data = {
    'Roll No': roll_numbers_list,
    'Pass': mlp.predict(x_test).tolist(),
}

In [ ]:
result = pd.DataFrame(data)

In [ ]:
result['Pass'][result['Pass'] == 0] = 'FAIL'
result['Pass'][result['Pass'] == 1] = 'PASS'

In [ ]:
result.to_csv('result_5.csv', index=False)